In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -q -o /kaggle/input/invasive-species-monitoring/train_labels.csv.zip

In [ ]:
train = pd.read_csv('./train_labels.csv')
train['name'] = '/kaggle/input/invasivespecies/train/'+train['name'].astype('str')+'.jpg'
train

In [ ]:
train['invasive'] = train['invasive'].astype('str')

In [ ]:
# valid 데이터셋 만들기
from sklearn.model_selection import train_test_split
X_train,X_valid = train_test_split(train,test_size=0.2,
                                  random_state=42,stratify=train['invasive'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator()
train_generator = idg.flow_from_dataframe(X_train,x_col='name',y_col='invasive',
                                         target_size=(200,200),batch_size=100)
valid_generator = idg.flow_from_dataframe(X_valid,x_col='name',y_col='invasive',
                                         target_size=(200,200),batch_size=100)
# 이미지 크게 하면 --> 점수개선에 도움 된다....
# => 40000개의 픽셀(200*200)로 더 디테일한 정보를 파악한다고 생각하자
# 너무 크면? --> 걸린 시간 + RAM소모량 대비 예측 점수향상이 높지 않음
# 정답은 아니지만 이상적인 사이즈 (300,300)
# 보통 target_size를 늘리면 batch_size를 줄여준다

In [ ]:
# 모델링 --> 직접쌓으면 비효율적일수 있음
# => 모델 가져와서 쓸것임
# vgg resnet & efficientnet
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB0
efn = EfficientNetB0(include_top=False,pooling='avg')
# include_top : 맨 위 출력층을 가져오지 않고 우리가 정답 클래스 개수를 정하겠다!
# pooling='avg'  --> Dense layer 이전의 flatten()의 역할을 해줌
# --> 그냥 flatten() 해주면 층에서 공부한 지역적인 요소가 많이 제거되므로 사용
efn.summary()

https://hoya012.github.io/blog/deeplearning-classification-guidebook-1/
### EfficientNet은 구글 이미지 검색

In [ ]:
model = Sequential()
model.add(efn) # efn 모델을 통으로 가져옴 --> summary()의 마지막 보면 밀집층의 정답 클래스 1000개
# --> 클래스 2개로 축소
model.add(Dense(2,activation='softmax'))
model.summary()  # efficientnetb0 부분이 많이 생략됨
# 맨 아래 Dense층 정답 클래스 2개로 축소
model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
model.fit(train_generator,validation_data=valid_generator,epochs=5)

In [ ]:
import glob
test = pd.DataFrame({'name':glob.glob('../input/invasivespecies/test/*')})
test

In [ ]:
test_generator = idg.flow_from_dataframe(test,x_col='name',y_col=None,
                                        target_size=(200,200),batch_size=100,
                                        class_mode=None,shuffle=False)

In [ ]:
result = model.predict(test_generator,verbose=1,workers=3)

In [ ]:
!unzip -q -o /kaggle/input/invasive-species-monitoring/sample_submission.csv.zip

In [ ]:
glob.glob('/kaggle/working/*')

In [ ]:
sub = pd.read_csv('/kaggle/working/sample_submission.csv')
sub['invasive'] = result[:,1]
sub['name'] = test['name'].apply(lambda x: x.split('/')[-1].split('.')[0])
sub

In [ ]:
sub.to_csv('result.csv',index=False)